In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time

In [ ]:
# Station and wharves dataframe
wharf_df = pd.read_csv('csv_inputs/wharf_info.csv')
# lines dataframe
line_df = pd.read_csv('csv_inputs/line_info.csv')
line_df['First_sailing'] = pd.to_datetime(line_df['First_sailing'], format='%H:%M')
# Wharf to wharf transit time dataframe
tt_df = pd.read_csv('csv_inputs/rebalancing_times.csv',index_col='From/To')
# Headways dataframe
headway_df = pd.read_csv('csv_inputs/headways.csv')
# vessels
vessel_df = pd.read_csv('csv_inputs/vessel_info.csv')
# charging berths dataframe
charging_berth = pd.read_csv('csv_inputs/charging_berths.csv')

# Simulation time parameters
initial_time = time(5,30)
period_length = 5  # mins
total_operation_hours = 6  # hours

# nc, Minimum num of crew breaks
nc = 1

# Dc, Crew break duration (fixed)
Dc = 9  # mins

# Tc, Maximum separation time for crew breaks
Tc = 6 * 60  # mins

# rv+, charging rate
rv_plus = 2100 * period_length / 60 / 1100  # kW*h/kWh --> %

# pc, Plugging/Unplugging time
pc = 2  # mins



# Functions

In [ ]:
def cal_time(period_num):
    """
    Calculates a new datetime based on an initial time and a given number of periods.
    
    Args:
    period_num (int): The period number to calculate the time for.
    initial_time (datetime.time): The starting time of the calculation.
    period_length (int): The length of each period in minutes.
    
    Returns:
    datetime.datetime: The calculated datetime.
    """
    # Convert initial_time to a datetime object with today's date
    initial_datetime = datetime.combine(datetime.today(), initial_time)
    
    # Calculate the total minutes to add
    added_time = timedelta(minutes=int((period_num - 1) * period_length))
    new_time = initial_datetime + added_time

    return new_time


In [ ]:
def cal_h(s, d, line): # copy from previous file
    """
    Calculate the s-th sailing time starting from time 'd' for a specified 'line'.

    This function retrieves headway periods for a specific line from a dataframe,
    constructs a list of sailing times starting from time 'd', and returns the s-th
    sailing time based on these intervals. It is used to project sailing schedules
    based on a specified headway and start day.

    Parameters:
    s (int): The order of the sailing time to be retrieved (1st, 2nd, etc.).
    d (int): The day from which sailing starts.
    line (int): The line number for which headway data is to be used.

    Returns:
    int or None: The s-th sailing time in terms of the number of time periods
                 since day 'd', or None if the index 's' is out of range or
                 an error occurs in processing.

    Note:
    The function assumes `period_length` as a global variable that denotes the
    length of each time period within the operational schedule.
    """

    try:
        # Retrieve the headway periods for the specified line and drop missing values
        h = headway_df[f'h{line}'].dropna().tolist()
        h_sd_ls = [d]  # Start the list with the initial day 'd'

        # Calculate subsequent sailing times based on headway periods
        for sailing_headway in h:
            num_time_period = int(sailing_headway // period_length + 1)  # round up
            h_sd_ls.append(h_sd_ls[-1] + num_time_period)

        # Return the s-th sailing time if within bounds
        if s-1 < len(h_sd_ls):
            return h_sd_ls[s-1]
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# Functions to replace and determine wharfs
def start_wharf(wharf):
    if wharf.isdigit():
        return Start_wharf.get(wharf, wharf)
    return wharf

def end_wharf(row):
    wharf = row['End_Wharf']
    if row['Task'].isdigit():
        line = int(row['Task'])
        timetoT = int(line_df[line_df['Line_No'] == line]['Time_underway_to_T'].iloc()[0])
        end_time = row['Start_Time'] + timetoT // period_length + 1
        matching_rows = Zp_df[(Zp_df['Line'] == line) & (Zp_df['Time'] == end_time)]
        if not matching_rows.empty:
            wharf = matching_rows['Wharf'].iloc()[0]
    return wharf

def end_time(row):
    if row['Task'].isdigit():
        line = int(row['Task'])
        duration = int(line_df[line_df['Line_No'] == line]['Line_duration'].iloc()[0])
        return (cal_time(row['Start_Time']) + timedelta(minutes=duration)).strftime('%H:%M')
    elif row['Task'] in ['Waiting', 'Charging']:
        return (cal_time(row['Start_Time']) + timedelta(minutes=5)).strftime('%H:%M')
    else:
        return (cal_time(row['Start_Time']) + timedelta(minutes=Dc)).strftime('%H:%M')

# Load results files

In [ ]:
def load_and_process_data(filepath, split_columns, value_columns):
    """
    Load data, remove unwanted characters, filter rows, split columns, and convert types.

    Args:
    filepath (str): Path to the CSV file.
    split_columns (list): List of columns names after splitting the 'Variable' column.
    value_columns (dict): Dictionary of columns to convert with their respective types.
    """
    df = pd.read_csv(filepath)
    df = df[df['Value'] == 1]
    df['Variable'] = df['Variable'].str.replace(r"[()']", "", regex=True)
    df[split_columns] = df['Variable'].str.split(',', expand=True)
    for column, dtype in value_columns.items():
        df[column] = df[column].astype(dtype)
    return df

# Load and process each DataFrame using the new function
z_df = load_and_process_data('output_files/z_wj_results.csv',['Wharf', 'Line'],{'Line': int})

Zp_df = load_and_process_data('output_files/Z_prime_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})

Z_df = load_and_process_data('output_files/Z_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})



In [ ]:
# start stations from line and wharf data
Start_S = dict(zip(line_df['Line_No'].astype(str), line_df['O']))
Start_S.update(dict(zip(wharf_df['Wharf_No'], wharf_df['Station'])))

# end stations from line and wharf data
End_S = dict(zip(line_df['Line_No'].astype(str), line_df['T']))
End_S.update(dict(zip(wharf_df['Wharf_No'], wharf_df['Station'])))

# start wharfs from z_df
Start_wharf = dict(zip(z_df['Line'].astype(str), z_df['Wharf']))

# end wharfs from Zp_df will be determined later

In [ ]:
z_df

# Timetable

In [ ]:
# Load and preprocess x_df
x_df = load_and_process_data('output_files/x_ld_results.csv',['Line', 'Time'],{'Line': int, 'Time': int})


# List of lines from line_df and x_df
lines = line_df['Line_No'].unique().tolist()
solved_lines = x_df['Line'].unique().tolist()


def cal_timetable(line):
    # Check if the line exists and is solved
    if line in lines and line in solved_lines:
        # First sailing time and headways
        d = x_df.loc[x_df['Line'] == line, 'Time'].iloc[0]
        headways = headway_df[f'h{line}'].dropna().tolist()

        times, locs, wharfs = [], [], []
        
        # Calculate timings for all sailings including first and subsequent ones
        for s in range(len(headways)+1):
            period = cal_h(s+1, d, line)  # Assuming cal_h is defined somewhere
            sailing_time = cal_time(period)
            
            # Add sailing time, location and wharf
            times.append(sailing_time)
            locs.append(Start_S[str(line)])
            wharfs.append(z_df[z_df['Line'] == line]['Wharf'].iloc()[0])

            # Intermediate Stop
            Intermediate_stop = line_df[line_df['Line_No'] == line]['I'].iloc()[0]
            if pd.notna(Intermediate_stop):
                timetoI = int(line_df[line_df['Line_No'] == line]['Time_underway_to_I'].iloc()[0])
                arrival_time_I = sailing_time + timedelta(minutes=timetoI)
                times.append(arrival_time_I)
                locs.append(Intermediate_stop)
                wharfs.append(z_df[z_df['Line'] == line]['Wharf'].iloc()[1])
            
            # Terminal Stop
            timetoT = int(line_df[line_df['Line_No'] == line]['Time_underway_to_T'].iloc()[0])
            arrival_time_T = sailing_time + timedelta(minutes=timetoT)
            times.append(arrival_time_T)
            locs.append(End_S[str(line)])
            wharfs.append(Zp_df[(Zp_df['Line'] == line) & (Zp_df['Time'] == period + timetoT // period_length + 1)]['Wharf'].iloc()[0])

        # Format times and create the DataFrame
        formatted_times = [time.strftime('%H:%M') for time in times]
        timetable = pd.DataFrame({
            'Time': formatted_times,
            'Station': locs,
            'Wharf': wharfs
        })
        return timetable
    
    else:
        print('Line not exist or unsolved.')
        return None

cal_timetable(1)

In [ ]:
cal_timetable(5)

# Vessel itinerary

In [ ]:
# Load data
y_df = load_and_process_data('output_files/y_vjt_results.csv',['Vessel', 'Task', 'Start_Time'],{'Start_Time': int})

# Extract and clean up Wharf details
y_df['Start_Wharf'] = y_df['Task'].apply(lambda x: x.split('_')[-1].strip())
y_df['End_Wharf'] = y_df['Start_Wharf']

# Lookup Start and End Stations based on Wharfs
y_df['Start_Station'] = y_df['Start_Wharf'].apply(lambda x: Start_S.get(x, 'Unknown Station'))
y_df['End_Station'] = y_df['Start_Wharf'].apply(lambda x: End_S.get(x, 'Unknown Station'))

# Update Task based on specific keywords or conditions
y_df['Task'] = y_df['Task'].apply(
    lambda x: 'Charging' if x.strip().lower().startswith('phi') else
              'Crew pause' if x.strip().lower().startswith('cp') else
              x.strip() if x.strip().isdigit() else
              'Waiting'
)

# Initial transformations for Start_Wharf and End_Wharf
y_df['Start_Wharf'] = y_df['Start_Wharf'].apply(start_wharf)
y_df['End_Wharf'] = y_df['Start_Wharf']  # Initially, set End_Wharf as Start_Wharf

y_df['End_Wharf'] = y_df.apply(end_wharf, axis=1)
y_df['End_Time'] = y_df.apply(end_time, axis=1)

# Reorganize DataFrame columns for final output
y_df = y_df[['Vessel', 'Task', 'Start_Station', 'Start_Wharf', 'Start_Time', 'End_Station', 'End_Wharf', 'End_Time']]

# Function to calculate vessel itinerary
def cal_itinerary(vessel):
    itinerary = y_df[y_df['Vessel'] == vessel].sort_values('Start_Time')
    itinerary['Start_Time'] = itinerary['Start_Time'].apply(lambda x: cal_time(x).strftime('%H:%M'))
    itinerary.reset_index(inplace=True, drop=True)
    return itinerary

In [ ]:
import pickle
with open(f'pkl_files/6htest_xi_jj_results.pkl', 'rb') as f:
    xi_jj_results = pickle.load(f)
xi_jj_results[('phi_PB1',11)]

In [ ]:
cal_itinerary('FF6')

In [ ]:
# cal_itinerary('10M')

# Wharfs utilisations
